# Exhaustive Grid Search

El método de optimización provisto por [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) genera candidatos a partir de una grilla de valores de parámetros específicados en `param_grid`.

In [1]:
param_grid = [
  {"C": [1, 10, 100, 1000], "kernel": ["linear"]},
  {"C": [1, 10, 100, 1000], "gamma": [0.001, 0.0001], "kernel": ["rbf"]},
 ]

En este caso se especifican dos grillas que se deben explorar. Cuando se usa [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) se entrenan todas las posibles combinaciones de los valores de los parámetros y se retiene la mejor combinación.

## Ejemplo

In [2]:
from sklearn import datasets

digits = datasets.load_digits()

In [3]:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target == 8
print(f"El número de imágenes es {X.shape[0]} y cada imagen contiene {X.shape[1]} pixeles")

The number of images is 1797 and each image contains 64 pixels


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

tuned_parameters = [
    {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
]

grid_search = GridSearchCV(SVC(), tuned_parameters, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}])

In [6]:
grid_search.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [7]:
from sklearn.metrics import classification_report

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.99      1.00      1.00       816
        True       0.97      0.94      0.96        83

    accuracy                           0.99       899
   macro avg       0.98      0.97      0.98       899
weighted avg       0.99      0.99      0.99       899



# Randomized Parameter Optimization

[`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) implementa una búsqueda randomizada sobre los parámetros, donde cada configuración es muestreada sobre una distribución de parámetros posibles. Esto tiene dos principales beneficios:

- Se reduce el costo computacioanl
- Añadir parámetros que no tienen unfluencia en el rendimiento no reducen la eficiencia

Al igual que con `GridSearchCV` se debe especificar una grilla de parámetros. Adicionalmente, se debe especificar el número de iteraciones (`n_iter`). Para cada parámetro, se puede especificar la dsitribución posible de valores que puede tomar o una lista discreta de elecciones (que serán muestradas de manera uniforme).

In [8]:
import scipy
param_grid = {
    "C": scipy.stats.expon(scale=100),
    "gamma": scipy.stats.expon(scale=.1),
    "kernel": ["rbf"],
    "class_weight": ["balanced", None]
}

Para parámetros continuos, es importante espeficiar una distribución continua para tomar la mayor ventaja posible de la randomización. De esta manera, aumentar el número de itearciones llevará a una búsqueda más fina.

In [9]:
from sklearn.utils.fixes import loguniform
param_grid = {"C": loguniform(1e0, 1e3),
    "gamma": loguniform(1e-4, 1e-3),
    "kernel": ["rbf"],
    "class_weight":["balanced", None]
}

## Comparación entre GridSearchCV y RandomizedSearchCV

In [10]:
import numpy as np
from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

# Cargamos datos
X, y = load_digits(return_X_y=True, n_class=3)

# Clasificador
clf = SGDClassifier(loss="hinge", penalty="elasticnet", fit_intercept=True)


# Función para reportar métricas
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


# Parámetros
param_dist = {
    "average": [True, False],
    "l1_ratio": stats.uniform(0, 1),
    "alpha": loguniform(1e-2, 1e0),
}

# RandomizedSearch
n_iter_search = 15
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search, n_jobs=-1)

# Tiempo de ejecución
start = time()
random_search.fit(X, y)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)
report(random_search.cv_results_)

# Grilla completa
param_grid = {
    "average": [True, False],
    "l1_ratio": np.linspace(0, 1, num=10),
    "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}

# GridSearchCV
grid_search = GridSearchCV(clf, param_grid=param_grid, n_jobs=-1)

# Tiempo de ejecución
start = time()
grid_search.fit(X, y)

print(
    "GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_["params"]))
)
report(grid_search.cv_results_)

RandomizedSearchCV took 0.14 seconds for 15 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.996 (std: 0.005)
Parameters: {'alpha': 0.05925317736009369, 'average': False, 'l1_ratio': 0.3410759198936478}

Model with rank: 2
Mean validation score: 0.993 (std: 0.004)
Parameters: {'alpha': 0.0167870153407712, 'average': False, 'l1_ratio': 0.3908249420788895}

Model with rank: 3
Mean validation score: 0.983 (std: 0.018)
Parameters: {'alpha': 0.034120673566747355, 'average': False, 'l1_ratio': 0.05693371351147103}

GridSearchCV took 0.36 seconds for 60 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.994 (std: 0.007)
Parameters: {'alpha': 0.01, 'average': False, 'l1_ratio': 0.3333333333333333}

Model with rank: 2
Mean validation score: 0.989 (std: 0.011)
Parameters: {'alpha': 0.1, 'average': False, 'l1_ratio': 0.0}

Model with rank: 3
Mean validation score: 0.989 (std: 0.004)
Parameters: {'alpha': 1.0, 'average': False, 'l1_ratio': 0.0}



# Bayesian Hyperparameter Optimization

In [19]:
import numpy as np
import sklearn.gaussian_process as gp

from scipy.stats import norm
from scipy.optimize import minimize

def expected_improvement(x, gaussian_process, evaluated_loss, greater_is_better=False, n_params=1):
    """ expected_improvement
    Expected improvement acquisition function.
    Arguments:
    ----------
        x: array-like, shape = [n_samples, n_hyperparams]
            The point for which the expected improvement needs to be computed.
        gaussian_process: GaussianProcessRegressor object.
            Gaussian process trained on previously evaluated hyperparameters.
        evaluated_loss: Numpy array.
            Numpy array that contains the values off the loss function for the previously
            evaluated hyperparameters.
        greater_is_better: Boolean.
            Boolean flag that indicates whether the loss function is to be maximised or minimised.
        n_params: int.
            Dimension of the hyperparameter space.
    """

    x_to_predict = x.reshape(-1, n_params)

    mu, sigma = gaussian_process.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    # In case sigma equals zero
    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        expected_improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)
        expected_improvement[sigma == 0.0] == 0.0

    return -1 * expected_improvement


def sample_next_hyperparameter(acquisition_func, gaussian_process, evaluated_loss, greater_is_better=False,
                               bounds=(0, 10), n_restarts=25):
    """ sample_next_hyperparameter
    Proposes the next hyperparameter to sample the loss function for.
    Arguments:
    ----------
        acquisition_func: function.
            Acquisition function to optimise.
        gaussian_process: GaussianProcessRegressor object.
            Gaussian process trained on previously evaluated hyperparameters.
        evaluated_loss: array-like, shape = [n_obs,]
            Numpy array that contains the values off the loss function for the previously
            evaluated hyperparameters.
        greater_is_better: Boolean.
            Boolean flag that indicates whether the loss function is to be maximised or minimised.
        bounds: Tuple.
            Bounds for the L-BFGS optimiser.
        n_restarts: integer.
            Number of times to run the minimiser with different starting points.
    """
    best_x = None
    best_acquisition_value = 1
    n_params = bounds.shape[0]

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, n_params)):

        res = minimize(fun=acquisition_func,
                       x0=starting_point.reshape(1, -1),
                       bounds=bounds,
                       method='L-BFGS-B',
                       args=(gaussian_process, evaluated_loss, greater_is_better, n_params))

        if res.fun < best_acquisition_value:
            best_acquisition_value = res.fun
            best_x = res.x

    return best_x


def bayesian_optimisation(n_iters, sample_loss, bounds, x0=None, n_pre_samples=5,
                          gp_params=None, random_search=False, alpha=1e-5, epsilon=1e-7):
    """ bayesian_optimisation
    Uses Gaussian Processes to optimise the loss function `sample_loss`.
    Arguments:
    ----------
        n_iters: integer.
            Number of iterations to run the search algorithm.
        sample_loss: function.
            Function to be optimised.
        bounds: array-like, shape = [n_params, 2].
            Lower and upper bounds on the parameters of the function `sample_loss`.
        x0: array-like, shape = [n_pre_samples, n_params].
            Array of initial points to sample the loss function for. If None, randomly
            samples from the loss function.
        n_pre_samples: integer.
            If x0 is None, samples `n_pre_samples` initial points from the loss function.
        gp_params: dictionary.
            Dictionary of parameters to pass on to the underlying Gaussian Process.
        random_search: integer.
            Flag that indicates whether to perform random search or L-BFGS-B optimisation
            over the acquisition function.
        alpha: double.
            Variance of the error term of the GP.
        epsilon: double.
            Precision tolerance for floats.
    """

    x_list = []
    y_list = []

    n_params = bounds.shape[0]

    if x0 is None:
        for params in np.random.uniform(bounds[:, 0], bounds[:, 1], (n_pre_samples, bounds.shape[0])):
            x_list.append(params)
            y_list.append(sample_loss(params))
    else:
        for params in x0:
            x_list.append(params)
            y_list.append(sample_loss(params))

    xp = np.array(x_list)
    yp = np.array(y_list)

    # Create the GP
    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)

    for n in range(n_iters):

        model.fit(xp, yp)

        # Sample next hyperparameter
        if random_search:
            x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
            ei = -1 * expected_improvement(x_random, model, yp, greater_is_better=True, n_params=n_params)
            next_sample = x_random[np.argmax(ei), :]
        else:
            next_sample = sample_next_hyperparameter(expected_improvement, model, yp, greater_is_better=True, bounds=bounds, n_restarts=100)

        # Duplicates will break the GP. In case of a duplicate, we will randomly sample a next query point.
        if np.any(np.abs(next_sample - xp) <= epsilon):
            next_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], bounds.shape[0])

        # Sample loss for new set of parameters
        cv_score = sample_loss(next_sample)

        # Update lists
        x_list.append(next_sample)
        y_list.append(cv_score)

        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

    return xp, yp

In [20]:
from sklearn.datasets import make_classification
data, target = make_classification(
    n_samples=2500,
    n_features=45,
    n_informative=15,
    n_redundant=5
)

In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

def sample_loss(params):
    C = params[0]
    gamma = params[1]
    model = SVC(C=10**C, gamma=10**gamma, random_state=42)
    return cross_val_score(
        estimator=model,
        X=data,
        y=target,
        scoring="roc_auc",
        cv=3
    ).mean()

In [22]:
bounds = np.array([[-4, 4], [-4, 4]])
bounds.shape

(2, 2)

In [23]:
x, y = bayesian_optimisation(100, sample_loss=sample_loss, bounds=bounds)

C:\Users\eavil\AppData\Local\Temp\ipykernel_13440\3311411319.py:71: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(fun=acquisition_func,
C:\Users\eavil\AppData\Local\Temp\ipykernel_13440\3311411319.py:71: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(fun=acquisition_func,
C:\Users\eavil\AppData\Local\Temp\ipykernel_13440\3311411319.py:71: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(fun=acquisition_func,
C:\Users\eavil\AppData\Local\Temp\ipykernel_13440\3311411319.py:71: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Current

In [24]:
x.shape

(105, 2)

In [25]:
print(x)

[[-1.11311193  0.08192189]
 [ 3.23111412  2.04693181]
 [-2.53394313  2.84729501]
 [-3.07084965 -2.84202131]
 [ 2.79061866  0.75087713]
 [-3.52566671 -3.57886693]
 [-2.07394466 -4.        ]
 [-2.74791333 -3.34440916]
 [-4.         -2.47583973]
 [ 0.62033695 -1.28000763]
 [ 0.77011942 -2.38970483]
 [-2.92151005 -3.63030349]
 [ 2.70859517  0.17579646]
 [ 0.275219   -2.03979529]
 [-3.4313563  -2.63084344]
 [-1.77501554 -1.9898536 ]
 [ 2.59561875  1.39098092]
 [ 1.91450531 -3.80726848]
 [ 0.44606508  2.3435803 ]
 [-1.85074352  2.11881538]
 [-1.31998467 -2.92526907]
 [-2.9751077  -3.37001655]
 [ 3.06289778 -1.44332408]
 [ 3.04839968 -2.54705664]
 [ 2.08878743 -2.1974707 ]
 [ 4.         -2.15647757]
 [ 1.74285299  2.76675421]
 [-1.15235464 -3.14123376]
 [ 3.20564851 -1.94943233]
 [-3.83828503 -3.02885133]
 [-2.18928454 -3.22463925]
 [ 3.03639217  2.83591595]
 [ 1.75621321 -3.85519805]
 [-0.82733789  0.04113037]
 [-2.59314116  2.09973766]
 [-2.35433662 -2.58811708]
 [-3.19351741 -0.06934526]
 

In [26]:
y.shape

(105,)

In [27]:
print(y)

[0.5        0.5        0.5        0.87731364 0.5        0.86371608
 0.86155831 0.86596795 0.89374005 0.96957144 0.99207615 0.86329369
 0.5        0.99212179 0.88585107 0.92511444 0.5        0.98317956
 0.5        0.5        0.93396073 0.86574337 0.97601042 0.99043128
 0.99208769 0.99219119 0.5        0.93266496 0.99188342 0.87166951
 0.86766691 0.5        0.97991572 0.5        0.5        0.88794762
 0.5        0.98964432 0.5        0.95312288 0.90088651 0.5
 0.96017341 0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.9770323  0.96712056 0.5        0.50160256
 0.94331328 0.94053973 0.96189772 0.93876963 0.96326173 0.92572785
 0.99173272 0.5        0.99010024 0.96494001 0.5        0.5
 0.5        0.5        0.5        0.95790619 0.97636195 0.97789472
 0.60075262 0.98744501 0.5        0.5        0.5        0.96206293
 0.94488843 0.5        0.94694237 0.98997638 0.5        0.88387356
 0.89867038 0.98955018 0.99190852 0.95644081 0.5        0.96269124
 0.5        0.5 

In [28]:
print(y.max())
print(x[y.argmax(), :])

0.9921911936773732
[ 4.         -2.15647757]


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
best_params = x[y.argmax(), :]
model = SVC(C=10**best_params[0], gamma=10**best_params[1], random_state=42)
model.fit(X_train, y_train)
accuracy_score(y_test, model.predict(X_test))

0.968

In [30]:
worst_params = x[y.argmin(), :]
model2 = SVC(C=10**worst_params[0], gamma=10**worst_params[1], random_state=42)
model2.fit(X_train, y_train)
accuracy_score(y_test, model2.predict(X_test))

0.466